In [ ]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time


def test_convergence(x):
    n,r = x

    nodes = []
    
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM(), simulation=False)
        node.start()
        nodes.append(node)

    print("a")
        
    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port,full=True)
        time.sleep(.1)
        print("a")
        

    time.sleep(3)     
        
    # Check if they are connected
    for node in nodes:
        assert len(node.neightbors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=0)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1)  
    
    return nodes
        
nodes = test_convergence((4,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:49545
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:33419
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:34551
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:45773
a


In [7]:
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
import threading

nodes = []
        
def test_node_down_on_learning(n):

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port,full=True)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightbors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=2,epochs=0)

    # Stopping node
    time.sleep(7)
    nodes[1].stop()

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break

    for node in nodes:
        node.stop()


nodes = []
for _ in range(6):
    test_node_down_on_learning(4)
    break


Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:58421
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:59085
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:36879
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:54231
INFO:root:127.0.0.1:59085 Connected to 127.0.0.1:58421
INFO:root:127.0.0.1:58421 Conexión aceptada con 127.0.0.1:59085
INFO:root:127.0.0.1:36879 Connected to 127.0.0.1:59085
INFO:root:127.0.0.1:59085 Conexión aceptada con 127.0.0.1:36879
INFO:root:127.0.0.1:58421 Connected to 127.0.0.1:36879
INFO:root:127.0.0.1:36879 Conexión aceptada con 127.0.0.1:58421
INFO:root:127.0.0.1:54231 Connected to 127.0.0.1:36879
INFO:root:127.0.0.1:36879 Conexión aceptada con 127.0.0.1:54231
INFO:root:127.0.0.1:58421 Connected to 127.0.0.1:54231
INFO:root:127.0.0.1:59085 Connected to 127.0.0.1:54231
INFO:root:127.0.0.1:54231 Conexión aceptada con 127.0.0.1:58421
INFO:root:127.0.0.1:54231 Conexión aceptad

INFO:root:(127.0.0.1:54231) Round 1 of 2 finished.
INFO:root:(127.0.0.1:58421) Self Vote: [('127.0.0.1:58421', 76), ('127.0.0.1:54231', 476), ('127.0.0.1:36879', 156)]
INFO:root:(127.0.0.1:36879) Round 1 of 2 finished.
INFO:root:(127.0.0.1:54231) Sending train set vote.
INFO:root:(127.0.0.1:58421) Waiting other node votes.
INFO:root:(127.0.0.1:36879) Sending train set vote.
INFO:root:(127.0.0.1:54231) Self Vote: [('127.0.0.1:36879', 180), ('127.0.0.1:59085', 240), ('127.0.0.1:58421', 193)]
INFO:root:(127.0.0.1:36879) Self Vote: [('127.0.0.1:58421', 23), ('127.0.0.1:59085', 319), ('127.0.0.1:54231', 71)]
INFO:root:(127.0.0.1:54231) Waiting other node votes.
INFO:root:(127.0.0.1:36879) Waiting other node votes.
INFO:root:(127.0.0.1:58421) Timeout for vote agregation. Missing votes from {'127.0.0.1:59085'}
FUNCTION NOT IMPLEMETED
INFO:root:(127.0.0.1:36879) Timeout for vote agregation. Missing votes from {'127.0.0.1:59085'}
FUNCTION NOT IMPLEMETED
INFO:root:(127.0.0.1:54231) Timeout for v

In [1]:
import pytest
from p2pfl.base_node import BaseNode
from p2pfl.communication_protocol import CommunicationProtocol
import time
import pytest
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
from p2pfl.settings import Settings


def four_nodes():
    n1 = Node(MLP(),MnistFederatedDM())
    n2 = Node(MLP(),MnistFederatedDM())
    n3 = Node(MLP(),MnistFederatedDM())
    n4 = Node(MLP(),MnistFederatedDM())
    n1.start()
    n2.start()
    n3.start()
    n4.start()

    return n1,n2,n3,n4

def test_gossip_heartbeat():

    n1.connect_to(n2.host,n2.port, full=False)
    n2.connect_to(n3.host,n3.port, full=False)
    n3.connect_to(n4.host,n4.port, full=False)

    time.sleep(2) # Wait for asincronity    

    assert len(n1.get_network_nodes()) == len(n2.get_network_nodes()) == len(n3.get_network_nodes()) == len(n4.get_network_nodes()) == 3

    
    n1.set_start_learning(rounds=2,epochs=0)    

    while all([n1.round is not None for n in [n1,n2,n3,n4]]):
        time.sleep(0.1)

    for n in [n1,n2,n3,n4]:
        n.stop()
        time.sleep(0.1)

n1, n2, n3, n4 = four_nodes()
test_gossip_heartbeat()

Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:37107
INFO:root:Nodo listening at 127.0.0.1:37339
INFO:root:Nodo listening at 127.0.0.1:40133
INFO:root:Nodo listening at 127.0.0.1:55395
INFO:root:127.0.0.1:37107 Connected to 127.0.0.1:37339
INFO:root:127.0.0.1:37339 Conexión aceptada con 127.0.0.1:37107
INFO:root:127.0.0.1:37339 Connected to 127.0.0.1:40133
INFO:root:127.0.0.1:40133 Conexión aceptada con 127.0.0.1:37339
INFO:root:127.0.0.1:40133 Connected to 127.0.0.1:55395
INFO:root:127.0.0.1:55395 Conexión aceptada con 127.0.0.1:40133
INFO:root:(127.0.0.1:37107) Broadcasting start learning...
INFO:root:(127.0.0.1:37339) Waiting initialization.
INFO:root:(127.0.0.1:37107) Waiting initialization.
INFO:root:(127.0.0.1:37107) Gossiping model initialization.
INFO:root:(127.0.0.1:37107) Gossiping model to 1 train set nodes.
INFO:root:(127.0.0.1:40133) Waiting initializa

INFO:root:(127.0.0.1:37339) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:37107) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:55395) Model added (1/4) from ['127.0.0.1:55395']
INFO:root:(127.0.0.1:55395) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:55395) Model added (2/4) from ['127.0.0.1:40133']
INFO:root:(127.0.0.1:40133) Model added (2/4) from ['127.0.0.1:37339']
INFO:root:(127.0.0.1:40133) Model added (3/4) from ['127.0.0.1:55395']
INFO:root:(127.0.0.1:55395) Model added (3/4) from ['127.0.0.1:37107']
INFO:root:(127.0.0.1:37339) Model added (2/4) from ['127.0.0.1:40133']
INFO:root:(127.0.0.1:55395) Model added (4/4) from ['127.0.0.1:37339']
INFO:root:(127.0.0.1:55395) Agregating models.
INFO:root:(127.0.0.1:40133) Model added (4/4) from ['127.0.0.1:37107']
INFO:root:(127.0.0.1:40133) Agregating models.
INFO:root:(127.0.0.1:37107) Model added (3/4) from ['127.0.0.1:55395', '127.0.0.1:40133']
DEBUG:root:(127.0.0.1:37107) Can't add a model tha

DEBUG:root:Closed connection: 127.0.0.1:40133
DEBUG:root:(127.0.0.1:55395) (NodeConnection Loop) Exception: [Errno 104] Connection reset by peer
DEBUG:root:Closed connection: 127.0.0.1:55395
INFO:root:(127.0.0.1:40133) Evaluating...
INFO:root:(127.0.0.1:40133) Training...
INFO:root:(127.0.0.1:55395) Evaluating...
INFO:root:(127.0.0.1:40133) Model added (1/4) from ['127.0.0.1:40133']
INFO:root:(127.0.0.1:55395) Training...
INFO:root:(127.0.0.1:40133) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:55395) Model added (1/4) from ['127.0.0.1:55395']
INFO:root:(127.0.0.1:55395) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:40133) Model added (2/4) from ['127.0.0.1:55395']
INFO:root:(127.0.0.1:55395) Model added (2/4) from ['127.0.0.1:40133']
INFO:root:(127.0.0.1:37339) Votes: {'127.0.0.1:55395', '127.0.0.1:40133', '127.0.0.1:37107'} of {'127.0.0.1:37339', '127.0.0.1:55395', '127.0.0.1:40133', '127.0.0.1:37107'}
INFO:root:(127.0.0.1:37107) Votes: {'127.0.0.1:37339', '

INFO:root:(127.0.0.1:37339) Gossiping model to 3 train set nodes.
INFO:root:(127.0.0.1:37107) Gossiping model to 3 train set nodes.
ERROR:root:(127.0.0.1:55395) Tried to add a model while learning is not running
ERROR:root:(127.0.0.1:55395) Tried to add a model while learning is not running
ERROR:root:(127.0.0.1:40133) Tried to add a model while learning is not running
ERROR:root:(127.0.0.1:40133) Tried to add a model while learning is not running
INFO:root:(127.0.0.1:37339) Round 2 of 2 finished.
INFO:root:(127.0.0.1:37339) Evaluating...
INFO:root:(127.0.0.1:37339) Taining finished!!.
ERROR:root:(127.0.0.1:37339) Tried to add a model while learning is not running
INFO:root:(127.0.0.1:37107) Round 2 of 2 finished.
INFO:root:(127.0.0.1:37107) Evaluating...
INFO:root:(127.0.0.1:37107) Taining finished!!.
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 37107 y desconectándose de 3 nodos
DEBUG:root:Closed connection: 127.0.0.1:37107
DEBUG:root:Closed connection: 127.0.0.1:37107

In [35]:
from collections import defaultdict
import os
import json
from torch.utils.data import Dataset
from pytorch_lightning import LightningDataModule
import numpy
import torch
from torch.utils.data import DataLoader, Subset, random_split

class JSON_MNISTDataModule(LightningDataModule):
    def __init__(self, train_dataset, test_dataset, batch_size=32, num_workers=4, val_percent=0.1):
        super().__init__()
        train_samples = round(len(train_dataset)*(1-val_percent))
        self.train_dataset, self.val_dataset = random_split(train_dataset, [train_samples, len(train_dataset) - train_samples])
        self.test_dataset = test_dataset
        
        self.batch_size = batch_size
        self.num_workers = num_workers

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)
    
class JSON_MNIST(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, samples, labels):
        """
        Args:
            samples (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.samples = torch.tensor(samples) 
        self.samples = torch.reshape(self.samples, (-1, 1, 28, 28)) # channels, width, height

        self.labels = torch.tensor(labels)
        assert len(self.samples)==len(self.labels)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx): 
        # adding a channel (b/w) 
        return self.samples[idx],self.labels[idx]
        

    
def get_samples(data_dir):
    train_data_dir = os.path.join(data_dir, 'data', 'train')
    test_data_dir = os.path.join(data_dir, 'data', 'test')

    # recordar cargarse groups
    users, groups, train_data, test_data = read_data(train_data_dir, test_data_dir)
    
    result = []
    for u in users:
        return train_data[u]["x"]
    

def build_datamodules(data_dir):
    train_data_dir = os.path.join(data_dir, 'data', 'train')
    test_data_dir = os.path.join(data_dir, 'data', 'test')

    # recordar cargarse groups
    users, groups, train_data, test_data = read_data(train_data_dir, test_data_dir)
    
    result = []
    for u in users:
        result.append( 
            JSON_MNISTDataModule( 
                JSON_MNIST(train_data[u]["x"],train_data[u]["y"]),
                JSON_MNIST(test_data[u]["x"],test_data[u]["y"])
            )
        )
    
    return result
    
    
    
def setup_clients(data_dir, model=None, use_val_set=False):
    eval_set = 'test' if not use_val_set else 'val'
    train_data_dir = os.path.join(data_dir, 'data', 'train')
    test_data_dir = os.path.join(data_dir, 'data', eval_set)

    users, groups, train_data, test_data = read_data(train_data_dir, test_data_dir)

    return users, groups, train_data, test_data


def read_data(train_data_dir, test_data_dir):
    '''parses data in given train and test data directories
    assumes:
    - the data in the input directories are .json files with 
        keys 'users' and 'user_data'
    - the set of train set users is the same as the set of test set users
    
    Return:
        clients: list of client ids
        groups: list of group ids; empty list if none found
        train_data: dictionary of train data
        test_data: dictionary of test data
    '''
    train_clients, train_groups, train_data = read_dir(train_data_dir)
    test_clients, test_groups, test_data = read_dir(test_data_dir)

    assert train_clients == test_clients
    assert train_groups == test_groups

    return train_clients, train_groups, train_data, test_data



def read_dir(data_dir):
    clients = []
    data = defaultdict(lambda : None)

    files = os.listdir(data_dir)
    files = [f for f in files if f.endswith('.json')]
    for f in files:
        file_path = os.path.join(data_dir,f)
        with open(file_path, 'r') as inf:
            cdata = json.load(inf)
        clients.extend(cdata['users'])

        data.update(cdata['user_data'])

    clients = list(sorted(data.keys()))
    return clients, [], data



In [36]:
from pytorch_lightning import Trainer, LightningModule
import torch
from torch import nn
from torch.nn import functional as F
import pytorch_lightning as pl
from torchmetrics import Accuracy

class MLP(LightningModule):
    """
    Multilayer Perceptron (MLP) to solve MNIST with PyTorch Lightning.
    """

    def __init__(self, metric = Accuracy, lr_rate=0.001): # low lr to avoid overfitting
        
        # Set seed for reproducibility iniciialization
        seed = 666
        torch.manual_seed(seed)

        super().__init__()
        self.lr_rate = lr_rate
        self.metric = metric()

        # 10 clases
        self.l1 = torch.nn.Linear(28 * 28, 128)
        self.l2 = torch.nn.Linear(128, 256)
        self.l3 = torch.nn.Linear(256, 62)

    def forward(self, x):
        """
        """
        batch_size, channels, width, height = x.size()
        
        # (b, 1, 28, 28) -> (b, 1*28*28)
        x = x.view(batch_size, -1)
        x = self.l1(x)
        x = torch.relu(x)
        x = self.l2(x)
        x = torch.relu(x)
        x = self.l3(x)
        x = torch.log_softmax(x, dim=1)
        return x

    def configure_optimizers(self):
        """
        """
        return torch.optim.Adam(self.parameters(), lr=self.lr_rate)

    def training_step(self, batch, batch_id):
        """
        """
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        self.log("train_loss", loss, prog_bar=True)
        return loss 

    def validation_step(self, batch, batch_idx):
        """
        """
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(self(x), y)
        out = torch.argmax(logits, dim=1)
        metric = self.metric(out, y) 
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_metric", metric, prog_bar=True)    
        return loss

    def test_step(self, batch, batch_idx):
        """
        """
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(self(x), y)
        out = torch.argmax(logits, dim=1)
        metric = self.metric(out, y) 
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_metric", metric, prog_bar=True)
        return loss
        

datamodules = build_datamodules("/home/pedro/Downloads/leaf/data/femnist")
tr = Trainer(max_epochs=10, accelerator="auto", logger=None, enable_checkpointing=False, enable_model_summary=False) 
tr.fit(MLP(), datamodules[55])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [18]:
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import transforms
from torchvision.datasets import MNIST
from math import floor
from pytorch_lightning import LightningDataModule

mnist = MNIST("", train=True, download=True, transform=transforms.ToTensor())

In [19]:
sorted_indexes = mnist.targets.sort()[1]
print(len(mnist.targets))
sorted_indexes

60000


tensor([10523, 42456, 42448,  ..., 47987, 55730, 18628])

In [20]:
mnist.targets = mnist.targets[sorted_indexes]
mnist.data = mnist.data[sorted_indexes]


In [21]:
for i,l in mnist:
    print(l)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


...

6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6


7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7


8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8


8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9


9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9


In [22]:
print(len(mnist.targets))


60000


In [1]:
a = b'START_LEARNING 2 1 8899253089791249729\nMODEL_INITIALIZED\nPARAMS\x80\x04\x95\x87\x00\x00\x00\x00\x00\x00\x00]\x94(\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x80M\x10\x03\x86\x94h\x04\x8c\x05dtype\x94\x93\x94\x8c\x02f4\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B\x00 \x06\x00\\ \\\xbc \x81\x86\xbc\xe3)\xe3\xbcN\\\xe9\xbc\xbc\xc1\xb6\xbc\xb0\xff\x0c\xbd\xfc\x00E\xbc\x98\x0e\xf3\xbcnj\xab;r\xf9\x0f\xbd\x9e\x1a\x91\xbc\x01Yr;\xber\xdd<%\xbb\x03;J\xee\x92\xb9j.\x0c=\x81@\xbf;%\x9bj\xba(\x8d\xfe\xbc%\xa8\x92<sv\x8a\xbcJ\xf7\x19\xbc%\xcd\x819\x9e@\xef<e\x05\x0c\xbc\x1c\x94\x94<\xc1\x01\xc4\xbb\x16\xad\x0f\xbd\x88\xb8\xfd\xbc\xe7\x07\xb6<\x8a}\x88\xbc\xab\xb1\x00\xbd\x01\xdb`\xbc\xa5\x8fw\xbc\n\x1e\xe5<\x93\xe6\x02<J^\xe0\xbb%\xabF\xbb8\xe1\xe2\xbb\x83Q\n\xbdg7\x03=J`H\xba\x93\xed\xb2\xbc\x84\xde\x08\xbdE\xf7\x03\xbd\x01\xcb6;wR\x8e;n\xe1\xa4:~E\x92<\xb7\x17\x03\xba\x13\x19J\xbb\x00L\x97\xbc\xca\x05\xf0\xbbJ\xcau\xbc\x85\xdf=\xbc\x05\x91\xd6\xbc%\xd4M\xbbW\x90\xa0<w+\x10\xbcnCl<\xdcG\x88;Q\r\xcc<e[$\xbc\xa8M\xdf<3\x16\x1a<\xe52a<\xaa\x1a\xde\xbcD!\x00=j\xa1P<\x01BX<\x17\xcb\xa3\xbc7\x87H\xbb\x93\x8a\x08<\xf8\xfd\xe7\xbbg\x98\x96\xbc\xfa\xfc\x02\xbd\xca\xdal\xbb\xf3uJ\xbc\x9a\xd1\xc1\xbc\xe5\x93\xab;\xf8\x7f\\\xbc\x8aZ\x89\xbc \x89\x07<\x13\xd3\xd3<s\xa3:\xbcE\xa7u<\x0c\x9f\xf1\xbc@\xd7\x97;\x9bv\x0f=<\x9d\xaa\xbc\x03\x18\xa1\xbce\xf0\x9f\xbc\xd7\x10\x12<\xa0B)<\xae\x8a\x0f\xbdq\x1e\xe7\xbc\xdd^\x01\xbdU\x99\xe5<r\xce\t\xbdh\x11\xdd\xbc\xe3\x89\x86<\n\x97\x83;~]\xcb\xbc\xf0\x01\xa5<\xc1\xd4\xe6;\n\xc8\xa6<JW\x9d;e\x87\xa6<<~\x8a<3 l\xbc\xfc\xb8\x91\xbc\x1a\x9d\xd5\xbcn\x8f\xe9\xba3\xfb\x08\xbd7]\x1f;\xb5\x01\xea<|\x18><n&L;\x01W\xd4<\x8a\xe0\xf8\xbcn<\x1b;\xbeV\xbe\xbcE\xb4]\xbc\xf1%\xd5\xbc\x9a\x1e\x0f\xbd|p\xc9\xbc`\xd0\x11\xbd\\\x81\xa5\xbb\x81\xfdR<LO\xb1\xbc\xf8L{\xbc\xa5/T\xbcGt\xa8\xbcE\x05\x1e<\xac\xdf\xc7\xbc\xaaq`<A\x91\xd2<\x98G\xef<\xe0\xd6\x94<\x81#\xd6;\xb7\xe3\x8e\xbaj\xde\x0e=\x00\x92\x87;\xc0\xf4\x85;\xdcT\x99;\x88u\xdc<Xkv\xbc\x820\x11=\x0e\xc9\xe2\xbcJ\x86\xe2\xba\x8a\x10\x04\xbd\xb7\x859\xb9\xae\xa4\xac\xbc_\xbf\x06=\x9cc\x0b<\xae\xf2\x80\xbbnd\x81\xbb\xb8}\xee\xba%\x11\x81\xbb\xe2\x7f\x02\xbd\n\x8e\xfe<\x9a\x8c\x0c=\x817^\xbb\x13\x91\xb5\xbc\x15\xda\x98<\xb7[\x96:n\xb2g<G\x18\xa0\xbc\x00\x08\r<\xca\xad\xab\xbc~\x08\xbb\xbc\xd7K\xa8\xbcw\xacJ\xbc\xba\x92\x9d\xbc\xc8E\xf6\xbcS)\xd8;?\x04\x12\xbd\x1c\xdc\'\xbcE\x95\xdb<Z\x00\xc9<\xa7\xaa\x03=\xa5E9;\xee\xad=\xbc8\xe5\xf5;J\xc2\xfb:.*\xeb;\\\xe7\xdc;\xce\x066<\xee\x08`<\x8a@\x83\xbb%\\\x98<\x1c\xc9\xbe<\xae\x1b\xa0<\xa1\x15\xf6\xbc\xba]\n=\xdc!\xb8\xbc\xa6\xc1\x03=\xa1\xea\xfd<\x9a\xec\x96\xbc\x13\x81\xc3;uA\xeb\xbc>\x96\xb2<\x10\x90\xa6\xbcLm\xcb\xbc8\x11\xe2<\x01Q.\xbc\xb8\xf8\xdc<\xf3\xa9\xe5<:\x83\xcb<@d\x8c\xbcu%\xb2\xbc\xaa\xce6\xbc\xb0N\x90\xbc\x8a,0<\xba\x9a\xd1\xbc\xbc\xf0\x00<\xac\x1e\xe0<:\xef\x08=\x0e\xee\xd1\xbcG\xa8\x01\xbd\xce\xde\x10\xbc\xe5\xa3g<\xf7a\xa0;\x88\x8b\x0c\xbd\x9c \xde;EL]\xbcn\x9aT<\xb7;\xbf\xbb\xee\x1e\x7f;W\x84\xd5\xbc%\xf4\x0f\xbb!\x18\xd0<\x93\xbfC\xbb\x10\xff\x9a\xbc>/\xe9\xbc=#\x00\xbd\xd3?E<`\xd4\xa7<\xe6\x8e\x10=\'{\xbf\xbc\x9cv\xfd<\xae\xaa\xf1\xbb\xea\xa5\r=J6\x82\xbb\x84\xe0\x10=\xa1W\xf5\xbcn\n\xfa<\x97\x11\x00<\xd1b\xf4<\x9a\x9b\xcc<\x9em\x0f\xbd\xc3\x1b\x06=\xe6*\x08\xbd\xcaK\x99<\xb3Ey\xbc\xee\nL;nn\xa3:\x97\xd0\n=^\xa6\x84<\xca\xd8\xb4<\xa5d\xf6\xbc\x97\xca\t=\x93\xbe\xf9\xbb\xe5M\x93;*\x17\x04=\x15\xc7\xd6<*TS\xbc\xbaJ\xb5\xbc\xa51\xf7\xbb8Vb;\xa7\xe8\xaa<L~\xe2\xbc[\x81\x11\xbd\xd5\x10\xa3\xbc\x1c\x19R\xbc\xbaj\xd3<\x11\xae\xc4\xbc74B<\xee0h;\x8eX\x11\xbd\xc0}\xa6<\xca\xa5\xb1\xbb\xf8\x0b\xee\xbbeO\x94<J\xead:\x1e\xa5\xfe\xbc\xc0V\x8b\xbb|9*<q\xaa\xf5\xbc\xb3%0<\x04\xb5\x08=\xbe\x8f\x03\xbd\x80r%\xbb\x0c>\x91\xbc\x13\xde\xb0\xbb\xc1\x8b\xd2\xbc\xaa\xe2\x86<n\xd9\xdb<S\x10\x1f\xbcd\xeb\x01\xbdn@`\xbcn;\x98\xb8Z\x0e\xc2<\xdc\xb6\xd07E\xe0\x0b\xbd\\k\n<\xd2\xd6\t\xbd\xc3\xa2\xb2<@\x88\x00<\xc1\xbe\xe6\xbcS|\xfd;O/\x05=\x95\xec\xc8<\xbeY\xaf\xbcQ\x7f\xe7\xbc\xe5\xed\x89\xbcJ\xd2]\xb9%\xd6\xee\xbb7J\x12\xbb*\xa7~\xbc\xc1\x01Q<\xa1\x9c5\xbc.\xed\xc9;\n\xaf\xe6\xbc|\xe2\xba\xbc\xae{\x97<e\x9e\xf5;S8s\xbc\xe7\xc0\xcf\xbcU\xec\x01=\xca\x14\'\xbc%e\xd8;\x1e\xe1\xb3<\x1c\x18K\xbc\x83\x01\xfe<S\xbc\xb0<\xf8\xcak\xbc\x93\x89\x81:\xba\x04\x91\xbc\xd0\x08\x06=N?\xd8\xbc.8\xd8<\x97\x15\x9c\xbcN\xb2\xaa\xbcS\xbbD\xbcg\xf1\xc5<\xae\x8fp<\\\xf2\x01\xbdu\x0e\xca\xbc\x98h\xef<\x13\x0cB\xbc\xc5\x82\x9d\xbcW\xe3-\xbc\xdc\xe0P\xbaJ\xa7\xb0\xbcj\xa9\x10=\x93-a<\xd5\x9f\x8f<NO\x02<sH\x0e\xbd\x01\xc3^<.\r\xaa\xbc\xf3\xe1@<\x11c\xf4<\xb0\xd1\x8a<74\x93\xbc\xdcb\x16;\xd3\xf9N<|\xab\xf3<\xf3\x0b\xdd\xbc\xe5g\xb6\xbc\x8e\x9b\xd7<\xa3\xe3\x00\xbd\xb7\x19\x0f\xbb\xb7O\xa8<\xf7J\x11<ni\xed<\x05\xd1\t\xbc\xdc\x89\xbf\xbb\x1ec\xff\xbc7\xd8>\xbc\xce\xcc\xca\xbc\xb7=\x87\xba\xee\xbb\x1c\xbc\x0c{\xe8<\x84\x9e\x04\xbdJ\xbe\xce;\x1a\xeb\xb2\xbcu}\xcc<S\xdb\x8c\xbc\xf0\x19\xaa\xbcNre<n\xf2N;u\xea\xe8\xbc\n7\x89\xbb\x910\xb9\xbcJ\xee\x1f\xbb\xdc\x91\x96:\xf8D\xe1;E\x11\x12\xbc\xc3J\xed\xbcn\xa2L\xbb\xee\xcc\x94<>\x94\xfd<\x8c\x89\x03=\xa7|\x82<\xc5\xeb><\xa5L\xa2\xbc\xca\x83}<s1\xae<(\xc2\xf0<\\\x9d\x87\xbb\xae:\x99<\xf3\xaeI<a40\xbc\xf7\xeb\x84;{\xad\x0f=a\xfc\xdb<\xf0\x8b\x8f\xbc!\x0f\xdd<\xba\x0f\xba<\xd5u\x9a\xbc\xa3\xd9\x99\xbc\xf7\x8c\x11\xbd\xaab\x95\xbc\'2\x85<%\xed!\xbb\xf3\xa6\t\xbc\xb3+\xae<\xc1\xc2\xfb;\xca2\xb3\xbb\xd7CT<N\xfdi<\x80]\x90\xbcnp\xbf\xba\x13{|\xbc\xf7q\xb2<n[\xc57!\x99S\xbc\xe0\x87\x1f\xbc\x01\\V\xbb\xf5\xf0\xf8<3B\xf2<\xb5V\xf3<JJ\x9e;n"\xd0:\xd5h\xcd<\x93\xdf]<\x0eNF<\xf3\xb1\xd0\xbc\xa7\xdc\x9d<\xb8 \xe2\xbaK\xf8\x06=e7\x8a;\xf7\xe2\xce<X\xe0\xef\xbc\x93\x93\xf9\xbc \xdc\x17<A\xe0\xea;LD\x89\xbc\xea\x0f\xfb\xbc\xa5\xa5\x1b\xbbC\xb6\xf3<e\xbf\xa2\xbc\x85\x10\xdb<.\xe7\x8e;\x00\xa0\x90\xba\xaa7\xc7\xbc@\x81\x98<.\xe6\xb1<-\xb9\x0e=\xea\xe6o\xbc\xf7\x92\xc5\xbc\xa1g\xd9\xbc\xea]E\xbc\xca\xcd\x12\xbc\x1c\x93\x03<E\x99\x00<,_\xa9<\x01\xa9\xd5<\xce\n\x07=\xfc!\xe6<\xe1\x99\xbf<Z8\t=`\xc6\x02\xbd\x85Y\xb7<\'\xd5\xbd<\x93\xdc\xd0:\x9c\xae\xf8\xbb\xa3\x14\x82\xbc\xd8\xfe\x07=n\x0e\xff;\x81\xa2O\xbc\xae\xd4\xdc;\xca]\xd2\xbb~v\x92\xbc\x80\xe9\x9c<\x9c\x99\x1f\xbc\xc3r\xd6\xbc\xd3!D<\xa1\xd7\xe0\xbc\xdf{\x05\xbd\\\xd8&;\xb8v\x0f\xbd\x17r\xab<%=+<\xea\x80o\xbc\xea\x89q<\x15\x96\x9e<\xa0J\x0b\xbcX\xdfj\xbc\xca\x8d\xe6\xbc\x9c\xaf\xf1<\r\xa4\x10\xbd5\xa6\x00=n\x91V\xbc\r\xce\x00=\xfcJh\xbc\'\xdd\x9c\xbc\xdc\\k\xbb\n\xe0\xbe<Q\xf2\xde<\xd5\r\r\xbdN\xbbX<C\xfc\xf6<\xbe.\xab<7\xf1\x10\xbc\x0c\xf2\xe6\xbc\x13\xc1\x93\xbb\x13\x9f\xd5\xbc\x85\xc6\x96<\xee]\xc8<\x01\x02\xf9\xbb7\x1d\xd0<\x05\xf2a<\x836\xcc<\x1a;\xda<#\xe5\xf3\xbc\\St;J\x95\x03<\xd5I\xe4<A\x98\xba;\xb8\x8a\xe7:\x8d\xb3\x0b\xbdxX\x0c\xbd\x15\x0f\xa4\xbc!2><\xaeMI\xbcS\x9a\xf3<HT\xfb\xbcZ3\xe9\xbceu\xda\xbb.\xf6\xfa<\x04\xa1\x01\xbd\xee6(\xbb\xbf$\n=\xecA\xb1<\x13WJ\xbb\xfb\xef\x04\xbd.8\xec;w\x14\x9b\xbcJ\x17\x07\xbd\xa5n\xfc\xbc\xa5\xa4.;\xf32-\xbc\xb7\xa0\xbe\xbcL\xec\xfa<\xf3\x80\x9a\xbcn\x15\t;\xbf\xd5\x10\xbd\xdcb&\xbb\xb3|\xad\xbc*`\x8a<\x1e\xd2\xd2\xbcJ\x96\xae;\x90\xdd\x85\xbc\xb13\xc4<\x0edw\xbc\x81\xfe\xe5<\xd3\xc6z\xbc\xd39\x94;\x11\xfa\xe0\xbc\xb8l\xdf\xba%\x04\xf7\xbcW\x98\r\xbc\x11\x13\xe2\xbc6)\x07=\xdaZ\xb2\xbc\x1c\xe4\x8b\xbcJ\xf6\\<lO\x0b=\xa5\xfe\x92;\xee\x0c_<N\x80`\xbcS\xd1\xf5\xbb.M\x12\xbc\xde\x06\x08\xbd\xea\r<<\x05\x17>\xbc\xea\x9aU\xbc\xb7\x85)\xba\x1c\xdc\xcb<q\x1a\xde\xbcu\xb4\xe1\xbc\xde\xfd\xa5\xbc\xb7\x0e\xb0:\xd3\x0b\xc5\xbb\x00\xb6\x16\xbb\xa7@\xd4\xbc\x04|\x07\xbd\xf7\xd3\x8c;.\xf4j\xbc\xdck\x18\xbb\xe7r\n\xbd\xc0\x8c\x1f\xbcn#\xf0\xb9\xfa\xb0\r=3\xecx<{\xbc\x00\xbd\xd7C\xbe<\x01/\xf3<T\x92\x0e=\xa3\t\xbd\xbc\xd0\xd5\x97\xbc\xe8M\x0c=\xa7\xac\x9c<0\x10\x9e<\xa3\x10\xae<\xa5\xebx;\xde\x9e\xd2<~:\xaa\xbcn3\xee;\xb3\xbb\x86<J\x86\xc5<\xb83\xfe\xba\x87l\x86<\x83Q\xd5\xbc\xdc\xf9\xac<\x00w\xa2\xbbw\x90\xbe\xbb\xdc\xb1\x00\xbb>R\xf4\xbc\xe6K\x01=\xd8)`<!\x17R<\x8e9\xb1<6\xcd\r=\xca\x14\x10\xbd\xdc\xca\x88\xba\x91\x08\xac\xbc\x8av*<\xe0\xc0\x9c\xbc\xda\x97\xfc\xbc\x13t\xed\xbb\xc5\xa8\x9b<%lN;Z#\xcd<\xe0m\x0b<S\x92\xbd\xbc\x0eJX<\x90\x15\xa9<\xc5d\x91<\xc1\xb8\xab;!?S\xbc\xca\x1c\x98\xbc\x05n\x9a<\xee\x8c=;\xca\xab\xe2\xbc\x13\xe50;J{D\xbb\x9cF\xbc<\xdc\x92\xc3\xb9\x80"\xa2<\x05\x84\xd3<s*\x9e<\x93#F\xbbn\xdd#\xba\xe0\xd4\x00\xbd\x97k\xd7\xbcWbQ<B\xc1\x0e\xbd\xd2A\x0e\xbd\x93\xf8\xdd\xbb\xf1Q\xb0\xbc\x13n\x04\xbcNbB<\xda\x96\xde\xbcJ\xfd\x88;\xb7\xbd\x96\xb9\xba\xc8\xd7<J\x99\x9d:@\x13\xa8;.\x8a\xbf\xbc\x8e\x13v\xbc\xa7\xfd\xcb<p\r\x87\xbc*\xec\x8f\xbcQ\x9e\xc1<\x93tZ\xbbc\xa1\x90<4[\x00=\xe5u\xf1;JT\xf2:\x18\xa9\xdc\xbc\x9c\xe3\x05\xbdw\x85\xad;JJ\xee\xbb!FR<\xf7f\xce<E\xe2\x01<\xe5\x15\x87\xbb\xbcX\x0c=\x8a5\x8f<H\\\xf7<\x07\x8c\x8f\xbc\x05\xb2\xf8\xbc\xdc\xec\t\xbaJ\xcd\xcf\xba\xa3\xd1\xe5\xbcJ\xbfF\xbb\xfa\x18\xd9<\xc7h\xbb<J\xc6\xaf;\xae\xbc!<n\xee\xab<\x9c\x1a\x8d<J\x87\xd0\xbc\x95\x89\x05\xbd\x97\x86\x91<,\xa4\x96<s\xe13<J\xf3A<\x8e\x1e\x85\xbc\x13\x9e?<\xec\x19\xd1\xbc\xe8z\xe5\xbcJ9\x8c;\xca\x080\xbca\xe31<\xc1\xfea<\xf0m\x93<%.\x9f<\x8e\xe6\xb0<cQ\x06\xbd<\x01f<\xebx\x07=j\r!<\xb8Lk<\xf7\xaf\x91<|R\x85<A\xbeb<\xdc\x8d\x82\xbb\xce\x84\xe9\xbc\\\xfe\xe6<>\xb7\xd6\xbc\x13W\x90\xbb:(\xcb\xbc7\xe5\xaf\xbcg\x94\x95\xbc\xe7\xe7\x99<\xca\x19q;l\x87\xb6\xbc%\x055\xbb\xc51N\xbc>\xe8\x88<w\x1aJ<s\r\xb8\xbc\xccR\xab\xbc\x9a\xe5\xe8\xbc\x01..\xbb|\x9b\'<\xa0\xa9\x8c<\xb5\xed\xf4<\x95\xd0\x97<w\xe7\x8f\xbcW62\xbc\xf7\xf2\x01\xbd\x01\xb0\xed<\xce\x9d:\xbcU{\xcb<\xaa\xd1a\xbcn\x8b\xc6<{H\t=\xe5\x99\xca<\x81t{;Pa\x06=\x15#\xe7\xbc\xda\n\xe6<E\x0fy<\xdcM%\xbc\x9c\xa6\xb9\xbb\x0ebl\xbc\xec\xcb\xd2\xbcRn\x08\xbd\xfc\xe2\xdb<\x0e\xf1\xa5<\x1cp\xac\xbc\xc5\xcb&\xbc\\\x05\xdb<n\x82\xcc\xbc\xd3KX<\x13k\xa0\xbc\xbe\xf4\xb0<.I\xfe<J\x18-<*\xcf\xf2<%\xd5[:J\xe3\xce<wQ\x86<E*\xe8\xbc,\xa9\xbd<\xcay\xda\xbc\xc8\xcd\x10\xbd#\xdf\xc7\xbc5\xfb\x8c\xbc\x80\xdc\r<\x1c\x7f\xae\xbb\x95\xdc\x0f\xbdg\x15\xd3<\x8a\xd1\xff<-\xb7\x04\xbd`>\xaa\xbc\x93\xa4K\xbc*\xa2,\xbc\xa5J\xbb\xbb\xb8\x0fr\xbb%~J<\xf7\x15\xc1;c\x04\xcf\xbc\\\xfb\'\xbb\x81$\xc4;\xee\xe9\x17<\'\x0e\x06\xbd\xd8`\xe7<%\xbd\xd0<.\xe7\xf4<\xfaF\xa3\xbcn\x02\xb9\xbb\x90#\x02=\x1c\xecM\xbc\xb8)\xe99%)\xad:\xd3\xf9\xf9\xbb\x85\xde\x96\xbc\x96\xd0\x0f\xbdC=\xf1\xbc|b\xe4<\xc78\x83\xbc\xf5\xf2\xbd\xbc\x7f\xea\x08\xbd\xea\xe1\x12\xbc\xae}\x1c\xbcQ\xc0\xf1\xbcwN\xc0\xbbl\xe5\xc9\xbc\xef\xcf\x03=N%b<\xb7K\x8c\xbb|\xa0,<\x00=\x15\xbbn\xeaP;\xf5\xe0\xe2<\x93U\xe1:\n\x02N<\x01\x86P\xbc0\x02\xaa<~V\xa2<\n\xd1\x87;!,\xe5<\x00\xee\x14\xbaWG\x12\xbd\x93/9\xbc\xf8\x9d\x0b\xbd\xd3\xc6\x94;e|\x04\xbd\xc5Y\xd9<\x13%\xde<\xef*\x12=\x80\xe9\x00;Z\x07\xe8<|\xfa1\xbc\xca\x1b\xc9<%H\xac\xbc%\x07\x11=\x052\xfd\xbc\x8e\xbf!<U\xc9\xbc<a\xf9\xde\xbc\x0eY\xff\xbcn\xcd=\xbb\xa5\xf0-<.\x80\xe2\xbb8f}<\x01\x17M<\x01\x18\xbb9\xbc\x92\xc3\xbcw\xe8\xd9\xbb\x07\x01\x0b\xbd\xe3]\xe5<3\xb1><\xca\xcbh\xbcsR\xc9<J~\x9c<\xae\xf2!\xbc\x91%\xb2\xbc\xd3\xa3p<\x95\xa9\xcb<\xa5\xb4\xd8\xbb\\\xf9\xf2\xbb\xae\x07\xb8\xbcEB&<\xbe.\xaf\xbc<Ij<\xf5\xb7\xb6\xbc\x93\xa6\x07\xbdn\x1b\xe5\xb8\xb7\xfb\x94:\xdc\xbd\x0c<\x81\xfd\xe5;\x1a\x98\x8b\xbc\xea)\xad\xbc\xdc\xea\xe1\xb9g!\xd7\xbc\x8c\xb0\xce\xbc\xfa\xdd\xd9<\x01\xb3\xbb<\x00\xef\x93\xbb\xc1v\xed\xbb\x85N\xd8\xbc\xae{g<\x01\xe8\xb9\xbc\xd7\xc5\x18\xbcp\xd3\x98<\xb5\r\x99<\xdcX\x9b<\xfe\x17\xd9\xbc\x93\x9bZ;(8\xf7\xbc\xb2\xe1\r=\x18\x89x\xbc\xb4\n\x12=\xcc\x9b\xa8<\x85\xa9\x84<\\\xfd\t\xbd\x91:\xd2\xbc\xcaX\x03=\xb7I=:\xb7i\x83\xba%\x84\x93:\xb3\xc3\xe5<\xee\xba\x0e\xbd\xbcz\xf2\xbc8\x8c\xdd;\xae3a\xbc\xf5\x05\xb7<7\x02\xbb\xbcU\x90\x05\xbd\x85\xc7\xdd\xbc\x98!\x07\xbdp@\x88\xbc\xf0\x04\x8d<l\x04\xe5<\xc1\x97<<cZ\x83\xbc\x9c.J\xbc\x01\x1ap<\\\x89v\xbc\xdc\xed\xf0:\xee\x08"<\xaf\xb0\x00=\x05\xde\x17\xbcN\xb7W<\xec\x1d\xf1\xbcG\xec\xbe\xbc\xb5\x99\xe0\xbc\xf7y\xc2\xbbsw\x16\xbcq\xfb\xc3\xbc7\xb1V\xbc\xd59\x8e\xbc\xec\x86\t=J\x12U<\x07q\xa7<\x8a\x8a\x81\xbb0\x82\xa4\xbc\x93\xf6D:J\x06G;\x1c\x12\x91<\x81qg\xbb\xb7\xbc\x92<\x01\xc6\xe7\xbc\xb1\xfd\xd1<\xa8\x9d\xe4\xbc\xe7K\x03\xbd\x04\xe6\x01=m\xbf\x04\xbdS\x9f\xe2\xbc\x0e\xa3\xcc\xbc\xd7\xba\x08\xbc\x81\x16\xd9\xbc*\x12\x10<x?\x10\xbdN\xc8\xe8\xbc\xb7iH:\xce\x00D<\x95\x01\xe9\xbcAOd<\x14.\x0f\xbde\xbf[<\xa7$\r\xbd\x93\x89\x1d\xbc\x1c\x1c\xb4\xbb\xa7:\xc5<\xa5\xc0\xd8\xbb\xc1@\xbd;\xc1\xf1R\xbc\x1c\xea\xea\xbcn\xd5\x85<\xeaL <zA\xd4\xbcn\x0f\x14\xba\xf0I\xa2\xbc\xb3h\xb5<\xc4\x1d\x04\xbd\n\xe5.<l\xa5\xbe\xbcH\x85\x10=\\P\x15;\x13*\xfc\xbc\x05N\x7f<\xc1;\xb6;\xc3R\xe8<S\xc5\x0b\xbd%\x17\xed\xbc\xaeI\xcf\xbcJ\x9d\x96\xbae\xdb|<-\xc7\x0c=\xb7\xae\x85\xbaks\x11=\xab\x86\x0e\xbd\xba\xe2\xfe\xbc5f\xd3\xbc\xb7\xdc0<\xdc\xb6\x9c9\xba`\xeb\xbc\x8a\x0e\x10\xbc\'<\x90<QN\xc9\xbcA\xed\xd8\xbb\xb3\xe2\x81<\xb1\xa2\xcb\xbc\xdc\xc6G\xb9\xcdP\x00\xbd\xe7\xa6\x98<:\xc5\xd5<\xdae\xd8<q\x92\xf6<\xaa(\x88\xbc\xae\xb4q<\x93\xd89\xbbh@\xde<z=\x02=\x8c`\xb2\xbc%3\xa4:\x83\xb0\xc0\xbcA(C<\xf7s\xc8\xbbp\xfc\x9e<\x10\x1c\x81\xbc\xa1\xdd\xb9<\xf1\x91\xe4<\xca9X\xbc\xd3\xc6\xab;\xb8U`\xbcW\x03\xcb<\x1c{\x05=8&y\xbc\xa1\x10\xcc\xbc\x80\x92\x92;\xbc\x9a\x82\xbc\x93z\xe2<ns\xfd9\\\x8a\x18\xbc@\'\x07\xbc\xec\x05\x8a<\x85Q\xf7\xbc\x83\xfb\xc9<\x00\x04\x99\xb9\xb7d\x06\xbb\xbe\x9e\xb7\xbc\x1c\xd1\xb8;>\xb8\xc9<\xf5R\xe7\xbc\'j\xa4<m\xc6\x11\xbd\xa58\xc9\xbc\x80F\x1d<\xb8\x0cn;SI\xd1<\x93\x9b\xac<\x93\x1cC\xbaW\x15\n=\xfaN\x06\xbd\xa5\xde\x0c\xbcs\xe5\x08\xbds\x11\x10<7RL;5\xe4\x00=\xca[\x7f\xbb\x1c\xe0o<\xdc\xdb\xc0<\x8a\xde\xd5;\xea\xdd=<*\x075<\x81U\xb1;3m\x1e</\x84\n\xbd\xd5\xdd\xba\xbc<c(\xbcj"\xd2\xbc\xabi\t=\x9a\xc5\xdf<\x0cW\x02=\xe3\x1f\x8f\xbce\xd2\xb7<\xaa+@\xbcP\x95\x10\xbd\n\xbf\xb0\xbc\xccs\xcc\xbc%\xd9\xf0\xb8\xe1\x06\xb4<L\xad\xde\xbc\x1c\xb0\xd6\xbc\xae\xcd\xaa;\x93>\xc0\xbaK\xaa\x00\xbd\xd7\xae4<zE\xd5\xbcJ\x87\x9d:\x85/ \xbc~\xe2\t\xbd\x92\xa3\x02=\x01\xad\xdb\xba\xa5"\xc9\xbc\xe0q\x10\xbc\x8a\xee \xbcX\x96\xe2\xbc\x8a\x1dk<7\x84@;\xcc\x02\xc8\xbc\xc2\x1b\x11=\xf8\xaa\xe2<\xc6\x00\x0f=\xb1\xd5\xe6\xbc\xc8\x08\xec\xbc\n\xba\x94;:L\xc6<>\xf1\xa7<\xae\x0f\xc0\xbc\xe1\x9dj\xbcG\xfe\x9b\xbc\x8c\x8c\xa8\xbc}\xda\t=\xd3\xe9k<\n\xc5\x1d\xbc3e\x1d\xbcj\xbe\x11<\x93\x82-\xbc\xd3\x10\x01\xbcs\xb3\x9b<x\xf0w<\n\xf9\r\xbcJs\x06\xbb\xb0\xa9\x88<u#\x9b\xbc\x83\xd4\xe6\xbc\xce\\u\xbc\n\xff\x08\xbc\xa7+\xa5<\x10\x91\x0e=1\xe8\xe2\xbc\xda\x9a\x11\xbd<\x05\xcf<\xdc\xf0\x18\xbbJ\r\xa0\xbc\xf2\xfc\x02=N\xcd\x9b\xbc\xf5t\r\xbdL\xe0\xc4\xbc\xb5\x0e\x8b\xbcJ\x9e\xf0:\xfe\xa9\xc9\xbc.\xd5\x84\xbc\xba\xfd\xd7<%gu\xbc:\xf4\x98\xbc\x01\xc17<\x93~1:-\xec\r=\x9cB\xca\xbbl\xae\x97<\xf5\xd2\xec<\xfer\xc3<\xa5B\xa8<\xee\xdc\xa1\xbb\\\'\x82\xbcn\x80\xcc\xba\\\xe3\x0c\xbc\x9a\xe8\xf6<G\xc0\xc8<|\x11~<\x11$\xd7<\xe7\xbe\xcc<\x01E\xb1\xbc\x97u\xb5\xbc\x9c\x98\xf2<\x91\xac\xe0<\xf3\xa5\xc0<S\x04\xe4;c\xf7\xd7<e\xdd2<\xeePl;\x93q\xe6\xba\x033\x8f<n\xe7u\xbbJ%Y\xbc\n#\xa8<\xe7Y\x08\xbd\x13\x03w\xbc`\x9f\x04<\x01\xe7\xdd\xba\x05\xe6\xba<V\x88\x0c=\xb8te<\x9a\xd2\xa5<\xdcrQ\xbc\xa3{\x9d<\xaeu\xfc\xbb\xa5\xb8C<\xaa\xfdY\xbc%\r\x82\xban\xb8\xb9\xbb\xc5T\xa8\xbcc\x13\x88<PZ\x97<3\xfc\x9b<%\x12\x12<S\x9c\xa1<J\x89\x03<\xd3\xe4\x8c\xbbpA\xa6<*\xb5\x92\xbc*\x93Y\xbc\x1a\xe7\xd2\xbc.\xf0r\xbcS\xc6\x00\xbc\x9c5\xef\xbcL\x9e\x80<S$\x01\xbc\xb7\xd7\x11<J\xb0\xd8<\xb8K\xe0\xba\xd89\xfa\xbc\xac\xa0\xac<8F\xfc\xbc\xdc\x03\xd9\xbb\x8e\x81\xd8<5(\xad<*;&\xbc\xf7V\x91;\x81\x9f\xb8<\x15n\x91<5\x00\xd7<1\xff\xfc\xbc\x93\xb7\x8e;&\xd6\x01\xbd\x08H\xdf\xbc\xb7\xb7\x0c=N\x9dx<\x1e\xa0\xc5\xbc\x93\xd6\x82<\x0c\x87\x83<\xfab\xef\xbc\x8a\x85\xd9;\x13\xfeh;\xcaE\xda\xbc*\xe6\xda\xbc\xe1\xf3z\xbc\x9a\x83\x8e\xbc.\xb83\xbce\x84\xec\xbbzX\xd2<\x83&\xc4<\xc1\xc1\xca<\x87\xae\x8a<s\xa0T<\x81\xbf4\xbb~{\x90<S\xac\x9e\xbces\xa4<:\xd0\xe1\xbc`\x10$<!Y.<E`\xc3<ES\xb5\xbc#H\x9d<\xee\x14\x10\xbb\xb8\xf5\xe8\xb9\x93r\xca\xba\\\x19(\xbbA\xa4\xc7\xbc\xfc\x86f<\xbc:Y\xbc\xdcE\xe4\xba\x1aV\x88<\x13\x82\xdc;\xe0i\x12<\x01\x16\xb8<\xcaey;\xce\xf8\\<.\xf7\x0e=\x8a\xde\xaa\xbb\xdbB\x0e=\x97\x17\xbd\xbc\x83\x14\xda\xbc\xacW\x9b<\xd5\xb6\xb0\xbc\xae\x86\xf7; L#\xbc\xb79\x8e\xbaE8\x12\xbc\n\x19P\xbc\x01h\xec\xb9\n<\xf3\xbc\x85\xa9\xed<\x17\xdc\xc7\xbc\x8ee\x93\xbc\x05M\xe9\xbc\x1a\xad\x86<~]\xe9\xbc\x05Q\x0c\xbc\xee\xd0\xd1\xbb\xf3\x03\xcd\xbc\xb7\x8c\x83;\x93\x96\xc2\xbb\xae\x9b\xd1<>\xe3\r\xbdzT\xfd<n\r\x1d\xbc\x070\x0c=\xdc25:\xe1\xde3\xbc\x93\xee8<!\x91f<S\xc3%\xbcjZ\x8c<\x03\x94\x84\xbcJ4&\xbc(\xf9\xfa\xbc\n\xba\x03=s\xf5\xf0<\xbaA\x08=\xe3\xf9\xaa<\x9f\xf7\x05=n7\x1d\xbbw\x8b\xc1\xbc\x87\xad\xa9\xbc\xb4\x81\x0f=\x93\xcd\x8b\xbc\x93\x82u:<+\x02<|\xf4\x17<wz\x06=\xa59\x84;>\x96\x97\xbcJ\'\x82\xbb\xd3\x17\x07\xbd\x93O~<k\xd7\r\xbd\xee\xd3\xcc\xbb\xd5\xa0\xd8\xbc\x9aK\xda<\x18\x85`<\xd8@~\xbc\x9au\xe2<\x989\xfe\xbc\x1e0\xfc\xbc\xd8\xb9\xe0<\xe1\xcei<\xfc\xd6\xa4\xbc\xce(\xc6<>K\r=\xa1\xa0\xec<\x03\xd3\xb8<\xd3\xfb\xe6;\xba\xd1\xf8\xbc\x01\xb7\xc3<\xdc\xcc\x04\xbb\x05\xc3\xb5\xbc^\xca\x9d\xbc\xee.\x0c\xbc\x0e*4<\xd3I\x91;\xdc\xcc\x9c:\x17\x8b\xbe\xbc\x1a"\xa0<\xb8\xd5\xf2;A\xd8\xef;\xac\x10\xe1\xbc\xb3\xa1\x1a\xbc%\x8b\x8d\xbbGC\x0c=\xda\xce\xd5<\x17[:\xbc.\xfb\x91<\\ZI\xbc+\xfa\r=\re\x04=\xeah\xa0\xbc\x15\x06\x9c\xbc\xc3h\xb8<{\xfb\x02=\x90\xc9\x97<7MZ\xbc\n\x03U\xbc\x8a[M\xbc,\x03\x08\xbd\xda\xc7\xbe\xbc\'\x88\x04=aob\xbcX*\xe6<%\xc2\xce\xba\xe1a\xe6<\xa5\x16\xae<lm\xa5<#\'\xb6\xbc\xd3\x95\xbe\xbb\x81#\xbb\xbc\xbd\x8d\x00\xbdj:Y<\x81/o<\xda\xbc\xd5\xbcne\x0b:.k\x85;\x85j4\xbc\x9c\xc0\xff;\x17s\x0c=\x80z\x0f;\xd7\xbf\xaf\xbc\xe5\xd0\xd3<\x98\xe0\xdd\xbc\x1a%\xe3<<\xe8\xa7<\xae\xfb@<.\xed\xe6\xbc\x94\xb2\x08=\xd3\x1e\xf3\xbc\xd75\x99<8\xbe\xeb<j\xfa\x1a<\xa0%\x0f<\x13\xe9:<\\p\x8a;S\xc9\x88;\xb5\x99\x01=\xd3\xe6\xcc;\x80\x06\xa0<7\x9e\xd6\xbc\x97J(\xbc\x88\xdb\xe2\xbc\xb56\x08=\x80D\x93;Sa\xc0\xbc\x00\x08\x12:\xecV\xd3<\xa5^\x93\xbc\xea\\<\xbc:\xb7\xbb\xbc\xeaK\x89\xbccg\xf8<*\xf2|<%\xce\x01<\xb7\x08\x85<A2\xfe\xbcA{\xed;\x80\x04\x9c;\x1cgQ<U)\x0c=\x8a\xd2\xbd\xbb\x85\ns<eO\xd1\xbb\x9cn\xc3;ub\xd9<\xca\x87d\xbc\x93\xa2<;\xdf\x87\x0b=*\x0fc\xbc\xb7a\xa8:1\n\xb1<p\xfc\x81\xbc\x85\xc7&\xbc\x15\x04\xa0\xbc;\x0b\x0c=\xa5\xf8\xc0\xbb\x97\x0b\x1a\xbcx\x1b\xdf\xbcj\'9\xbc\n=\xa2;\xfa3\xcf<.\x88\xc8\xbc\x93\x90i\xba\xaa\x95\xb6\xbc\xb3\x8c\xdd<M\xff\x06=\\N0<S\xba\xd7\xbb\xd1>\xd6<w\xd9\xb1<e?\t=W\x95B<\xa5\xd9\x02\xbb\xdc\xd0:\xbb\x01lq\xbb\xb5\xe0\x03\xbd\xb7\xe2\x02\xbc\xd38\xd7\xbc\x8c\x04\xd3<,U\xd0\xbc\xf1P\xd1\xbc\x00\x95&;\xae\x85\xc7;\xb3\xc2\x01=\xccI\xc5<\xd5\x80\x9c<\xf7o\x11<S\x9b\xe8<\x83\xf4\x92\xbc\xd2\x8d\x07=\x01\xe0z;J\x05\x8b\xba:\x11\x81<\xec\xe0\xe3\xbc\xaa\xacV\xbcn-\xb6\xba\xa8U\xed\xbc\xb7\xe6\xc1\xbbz\x89\x92\xbc`\xec\x0e\xbd\x18\x89k<\xcb\xea\x07\xbdz\x85\xd8<\xa56H<\xd4\xa8\x10\xbd:\x1b\x9d<\x05\xb7\xd7<\\\xe6G\xbc.\xa2\xe9;\xee\xeb\x1b\xbb\x97\xffZ<b\x0c\t\xbd\xaa\xad\x0f\xbc\x81!\xb0<\xfc\xe5\x0e\xbd8\x98t;\x11\x83\xb7<\'\x13\x03\xbdei\xc9;%0\xad:\xfe+\xff<.\xc8\xf2<*B"\xbc\xf1\xf1\xdf<\xe7Q\x89\xbcn\xdf\xb9\xb9\xae\xc0\xbb;SU\x8d\xbb\xfc\xdd\x19\xbc%\r\xb5<\x15\x90\xf2<\x93$,;\x9eP\xfe\xbc\x1c\x121<\x01\xda\xbc\xbc\x9e\xbb\x98<J\x8a\xc6\xbc\xee\xd6\x0c;\x93\xd2(<\xcc\x0b\xd3\xbcA\x01E<\xd1K\xad\xbc\x00\xc8\x87:X\x8c\xfc\xbc\x8a\xe8\xe0\xbcS}S<\x8e\x888\xbc\xb7y\xae:\x8a \xb2\xbb\xb3C\x0c=e\xb0!<\x13[~\xbcsm\xde\xbc,\xff\x05=\xea\xfa\xc4\xbcA;\xbe\xbcj\x1dM\xbc\\.\x0c;L\x95\xfd<\x1eB\xa3\xbc\x8a\xd3\xc1<*~(<\xcc\xf2\x99\xbc\xe7\xe8\xcb\xbc\xa5D\xa2<\xaa\x8e\x01<\x0b\xc7\x11=\xa8b\x03\xbd\x93\x1b\x18;\xee\xbdK< \xd7\x0f\xbd\xb7\x06\xaf:u\xec\x0f=\xe1\xccl<\x01j\xcd;%\xdb\x18;7\x92\x83\xbb\x00p\'9\x81\xcfi;$\x81\x10=\xeeDY;%\x0e7\xbb<\xa2\xa4\xbc\xe1(7\xbc\xb8\x11\xf3\xbc\xd3\xb9\xe9\xbb\xdcC\xa4\xbb.H\x9e;j\xc1\xb5\xbcc\n\xa5\xbc:(\xe0\xbc>*\xbf<,5\xe2<\xb8\xc3v\xbc\x93\x90\x96\xbb\x8e\xda\xe8<\xe3\xef\xa7\xbc^\xa3\xca<\x1b\xab\x10\xbd\x9a\xde\xcb\xbc\x81.Q<5\xc7\xa5<z\xde\xb5<\xda\xb5\xa5\xbc%\xf1\x80:\n\x14\xaf<J\xbd\x91<\x81\xe4\xf4\xbc,m\xc8<\x87H\xaa<E\xd0\x1b\xbcn\xd5\xc0:*^\x99\xbc\xc7\xf6\xa9<\x93Z\x98\xba\xb7\x90=<w\x89\x1c<\x81\xa2j\xbb\x10\xa8\xa6<\x15x\xc2\xbc\xc0\x0f\x92\xbc\x85w\xc6\xbc\xd3\xf8\xa9\xbc\xa5\xc0\x89<\x93$"\xb8n\xdf\x06<\x93*\xd2;\xa0i\x02=\x1e]\xc2\xbc*\xf1\x0f<\xc3)\xa5\xbc\xe7\x14\r=:B\x8f\xbc\xec\x01\xec\xbc%\xc0\xcd<\xd1\xe6\xe8<\xae\xf5\x07=\xe3S\xb9\xbc\x8a\x96\xd9\xbc\xa1f\xd1\xbc\xf3\xb5!\xbc\x01+[\xbc77V\xbb\xee\x8c\xd9<\n\x0c\xa2\xbc\x93\xc5\xce\xbc%\xdd\xa9\xbc\x9d\xa4\x04=\xe547\xbc\x1bW\x01\xbd\xb8\xaf\xf9\xbb^\xc4\xb7<\xd3\x88\x99<\xdc\\\x92\xba\xa55\x89\xbc\\\x8e\xfa;S\x85C<\xdcD(;.\x0c\xcd;n9U\xba\xb7\x8d\x8f\xba\xac!\xf9\xbc\xdc\xc0\x9b\xbbSY\xda\xbc\x00\x86\x92<\xdc\x98"<s\xbeQ<\n\xe1\x08\xbd\xa3\x13\xd9<\x8cQ\xcc<S\xf9\xd9\xbc\xaa\xc3N<\x9c\xb6\xbe\xbc\xf3\xe6\xd5\xbc\x85\xc2+\xbcn\x91K:*\xc4q<\x80\x89\x02\xbc\xdc\x80\x8d\xbb\xf7\xee\x98\xbc\x13\xdf\xc5\xbcd.\x00\xbdS\xde\xe5<\xc0\xf5\x95\xbc\xb8F\xe9: \x86\x1a<\xeb\x1e\x02\xbd\xa5B\xc5\xbcGY\xc8<\xb7n7\xbbZA\x9a\xbc\x8a9M<\xd1\xea\xe7<\xeaE\xd2\xbc\x93K\x8e:\xae\x9b\xfb\xbb\x98\x1d\x0c=n\x91I\xbcJ \x99\xbc\x13\xcb\xaf;L\xe2\xf1\xbc\x01\xf73;\x01%\xdb\xbc\x00\x86!;\x9ch\x88\xbb1\xa3\xcf\xbc%\xd9\r\xbd@d\x11<\xae\xcb\xa8\xbc\x95\x04\xe9\xbc\xaa\xb3\x87\xbc\xa5vB\xbb\x07c\xd4\xbc\xee\xe7l<b\xd3\t=<\xbc\x7f\xbcN\xe9\x0f=\xa3'

In [4]:
from p2pfl.communication_protocol import CommunicationProtocol
header = CommunicationProtocol.PARAMS.encode("utf-8")
header

b'PARAMS'

In [5]:
a.find(header)


57

In [10]:
len(b'PARAMS\x80\x04\x95\x87\x00\x00\x00\x00\x00\x00\x00]\x94(\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x80M\x10\x03\x86\x94h\x04\x8c\x05dtype\x94\x93\x94\x8c\x02f4\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B\x00 \x06\x00\\ \\\xbc \x81\x86\xbc\xe3)\xe3\xbcN\\\xe9\xbc\xbc\xc1\xb6\xbc\xb0\xff\x0c\xbd\xfc\x00E\xbc\x98\x0e\xf3\xbcnj\xab;r\xf9\x0f\xbd\x9e\x1a\x91\xbc\x01Yr;\xber\xdd<%\xbb\x03;J\xee\x92\xb9j.\x0c=\x81@\xbf;%\x9bj\xba(\x8d\xfe\xbc%\xa8\x92<sv\x8a\xbcJ\xf7\x19\xbc%\xcd\x819\x9e@\xef<e\x05\x0c\xbc\x1c\x94\x94<\xc1\x01\xc4\xbb\x16\xad\x0f\xbd\x88\xb8\xfd\xbc\xe7\x07\xb6<\x8a}\x88\xbc\xab\xb1\x00\xbd\x01\xdb`\xbc\xa5\x8fw\xbc\n\x1e\xe5<\x93\xe6\x02<J^\xe0\xbb%\xabF\xbb8\xe1\xe2\xbb\x83Q\n\xbdg7\x03=J`H\xba\x93\xed\xb2\xbc\x84\xde\x08\xbdE\xf7\x03\xbd\x01\xcb6;wR\x8e;n\xe1\xa4:~E\x92<\xb7\x17\x03\xba\x13\x19J\xbb\x00L\x97\xbc\xca\x05\xf0\xbbJ\xcau\xbc\x85\xdf=\xbc\x05\x91\xd6\xbc%\xd4M\xbbW\x90\xa0<w+\x10\xbcnCl<\xdcG\x88;Q\r\xcc<e[$\xbc\xa8M\xdf<3\x16\x1a<\xe52a<\xaa\x1a\xde\xbcD!\x00=j\xa1P<\x01BX<\x17\xcb\xa3\xbc7\x87H\xbb\x93\x8a\x08<\xf8\xfd\xe7\xbbg\x98\x96\xbc\xfa\xfc\x02\xbd\xca\xdal\xbb\xf3uJ\xbc\x9a\xd1\xc1\xbc\xe5\x93\xab;\xf8\x7f\\\xbc\x8aZ\x89\xbc \x89\x07<\x13\xd3\xd3<s\xa3:\xbcE\xa7u<\x0c\x9f\xf1\xbc@\xd7\x97;\x9bv\x0f=<\x9d\xaa\xbc\x03\x18\xa1\xbce\xf0\x9f\xbc\xd7\x10\x12<\xa0B)<\xae\x8a\x0f\xbdq\x1e\xe7\xbc\xdd^\x01\xbdU\x99\xe5<r\xce\t\xbdh\x11\xdd\xbc\xe3\x89\x86<\n\x97\x83;~]\xcb\xbc\xf0\x01\xa5<\xc1\xd4\xe6;\n\xc8\xa6<JW\x9d;e\x87\xa6<<~\x8a<3 l\xbc\xfc\xb8\x91\xbc\x1a\x9d\xd5\xbcn\x8f\xe9\xba3\xfb\x08\xbd7]\x1f;\xb5\x01\xea<|\x18><n&L;\x01W\xd4<\x8a\xe0\xf8\xbcn<\x1b;\xbeV\xbe\xbcE\xb4]\xbc\xf1%\xd5\xbc\x9a\x1e\x0f\xbd|p\xc9\xbc`\xd0\x11\xbd\\\x81\xa5\xbb\x81\xfdR<LO\xb1\xbc\xf8L{\xbc\xa5/T\xbcGt\xa8\xbcE\x05\x1e<\xac\xdf\xc7\xbc\xaaq`<A\x91\xd2<\x98G\xef<\xe0\xd6\x94<\x81#\xd6;\xb7\xe3\x8e\xbaj\xde\x0e=\x00\x92\x87;\xc0\xf4\x85;\xdcT\x99;\x88u\xdc<Xkv\xbc\x820\x11=\x0e\xc9\xe2\xbcJ\x86\xe2\xba\x8a\x10\x04\xbd\xb7\x859\xb9\xae\xa4\xac\xbc_\xbf\x06=\x9cc\x0b<\xae\xf2\x80\xbbnd\x81\xbb\xb8}\xee\xba%\x11\x81\xbb\xe2\x7f\x02\xbd\n\x8e\xfe<\x9a\x8c\x0c=\x817^\xbb\x13\x91\xb5\xbc\x15\xda\x98<\xb7[\x96:n\xb2g<G\x18\xa0\xbc\x00\x08\r<\xca\xad\xab\xbc~\x08\xbb\xbc\xd7K\xa8\xbcw\xacJ\xbc\xba\x92\x9d\xbc\xc8E\xf6\xbcS)\xd8;?\x04\x12\xbd\x1c\xdc\'\xbcE\x95\xdb<Z\x00\xc9<\xa7\xaa\x03=\xa5E9;\xee\xad=\xbc8\xe5\xf5;J\xc2\xfb:.*\xeb;\\\xe7\xdc;\xce\x066<\xee\x08`<\x8a@\x83\xbb%\\\x98<\x1c\xc9\xbe<\xae\x1b\xa0<\xa1\x15\xf6\xbc\xba]\n=\xdc!\xb8\xbc\xa6\xc1\x03=\xa1\xea\xfd<\x9a\xec\x96\xbc\x13\x81\xc3;uA\xeb\xbc>\x96\xb2<\x10\x90\xa6\xbcLm\xcb\xbc8\x11\xe2<\x01Q.\xbc\xb8\xf8\xdc<\xf3\xa9\xe5<:\x83\xcb<@d\x8c\xbcu%\xb2\xbc\xaa\xce6\xbc\xb0N\x90\xbc\x8a,0<\xba\x9a\xd1\xbc\xbc\xf0\x00<\xac\x1e\xe0<:\xef\x08=\x0e\xee\xd1\xbcG\xa8\x01\xbd\xce\xde\x10\xbc\xe5\xa3g<\xf7a\xa0;\x88\x8b\x0c\xbd\x9c \xde;EL]\xbcn\x9aT<\xb7;\xbf\xbb\xee\x1e\x7f;W\x84\xd5\xbc%\xf4\x0f\xbb!\x18\xd0<\x93\xbfC\xbb\x10\xff\x9a\xbc>/\xe9\xbc=#\x00\xbd\xd3?E<`\xd4\xa7<\xe6\x8e\x10=\'{\xbf\xbc\x9cv\xfd<\xae\xaa\xf1\xbb\xea\xa5\r=J6\x82\xbb\x84\xe0\x10=\xa1W\xf5\xbcn\n\xfa<\x97\x11\x00<\xd1b\xf4<\x9a\x9b\xcc<\x9em\x0f\xbd\xc3\x1b\x06=\xe6*\x08\xbd\xcaK\x99<\xb3Ey\xbc\xee\nL;nn\xa3:\x97\xd0\n=^\xa6\x84<\xca\xd8\xb4<\xa5d\xf6\xbc\x97\xca\t=\x93\xbe\xf9\xbb\xe5M\x93;*\x17\x04=\x15\xc7\xd6<*TS\xbc\xbaJ\xb5\xbc\xa51\xf7\xbb8Vb;\xa7\xe8\xaa<L~\xe2\xbc[\x81\x11\xbd\xd5\x10\xa3\xbc\x1c\x19R\xbc\xbaj\xd3<\x11\xae\xc4\xbc74B<\xee0h;\x8eX\x11\xbd\xc0}\xa6<\xca\xa5\xb1\xbb\xf8\x0b\xee\xbbeO\x94<J\xead:\x1e\xa5\xfe\xbc\xc0V\x8b\xbb|9*<q\xaa\xf5\xbc\xb3%0<\x04\xb5\x08=\xbe\x8f\x03\xbd\x80r%\xbb\x0c>\x91\xbc\x13\xde\xb0\xbb\xc1\x8b\xd2\xbc\xaa\xe2\x86<n\xd9\xdb<S\x10\x1f\xbcd\xeb\x01\xbdn@`\xbcn;\x98\xb8Z\x0e\xc2<\xdc\xb6\xd07E\xe0\x0b\xbd\\k\n<\xd2\xd6\t\xbd\xc3\xa2\xb2<@\x88\x00<\xc1\xbe\xe6\xbcS|\xfd;O/\x05=\x95\xec\xc8<\xbeY\xaf\xbcQ\x7f\xe7\xbc\xe5\xed\x89\xbcJ\xd2]\xb9%\xd6\xee\xbb7J\x12\xbb*\xa7~\xbc\xc1\x01Q<\xa1\x9c5\xbc.\xed\xc9;\n\xaf\xe6\xbc|\xe2\xba\xbc\xae{\x97<e\x9e\xf5;S8s\xbc\xe7\xc0\xcf\xbcU\xec\x01=\xca\x14\'\xbc%e\xd8;\x1e\xe1\xb3<\x1c\x18K\xbc\x83\x01\xfe<S\xbc\xb0<\xf8\xcak\xbc\x93\x89\x81:\xba\x04\x91\xbc\xd0\x08\x06=N?\xd8\xbc.8\xd8<\x97\x15\x9c\xbcN\xb2\xaa\xbcS\xbbD\xbcg\xf1\xc5<\xae\x8fp<\\\xf2\x01\xbdu\x0e\xca\xbc\x98h\xef<\x13\x0cB\xbc\xc5\x82\x9d\xbcW\xe3-\xbc\xdc\xe0P\xbaJ\xa7\xb0\xbcj\xa9\x10=\x93-a<\xd5\x9f\x8f<NO\x02<sH\x0e\xbd\x01\xc3^<.\r\xaa\xbc\xf3\xe1@<\x11c\xf4<\xb0\xd1\x8a<74\x93\xbc\xdcb\x16;\xd3\xf9N<|\xab\xf3<\xf3\x0b\xdd\xbc\xe5g\xb6\xbc\x8e\x9b\xd7<\xa3\xe3\x00\xbd\xb7\x19\x0f\xbb\xb7O\xa8<\xf7J\x11<ni\xed<\x05\xd1\t\xbc\xdc\x89\xbf\xbb\x1ec\xff\xbc7\xd8>\xbc\xce\xcc\xca\xbc\xb7=\x87\xba\xee\xbb\x1c\xbc\x0c{\xe8<\x84\x9e\x04\xbdJ\xbe\xce;\x1a\xeb\xb2\xbcu}\xcc<S\xdb\x8c\xbc\xf0\x19\xaa\xbcNre<n\xf2N;u\xea\xe8\xbc\n7\x89\xbb\x910\xb9\xbcJ\xee\x1f\xbb\xdc\x91\x96:\xf8D\xe1;E\x11\x12\xbc\xc3J\xed\xbcn\xa2L\xbb\xee\xcc\x94<>\x94\xfd<\x8c\x89\x03=\xa7|\x82<\xc5\xeb><\xa5L\xa2\xbc\xca\x83}<s1\xae<(\xc2\xf0<\\\x9d\x87\xbb\xae:\x99<\xf3\xaeI<a40\xbc\xf7\xeb\x84;{\xad\x0f=a\xfc\xdb<\xf0\x8b\x8f\xbc!\x0f\xdd<\xba\x0f\xba<\xd5u\x9a\xbc\xa3\xd9\x99\xbc\xf7\x8c\x11\xbd\xaab\x95\xbc\'2\x85<%\xed!\xbb\xf3\xa6\t\xbc\xb3+\xae<\xc1\xc2\xfb;\xca2\xb3\xbb\xd7CT<N\xfdi<\x80]\x90\xbcnp\xbf\xba\x13{|\xbc\xf7q\xb2<n[\xc57!\x99S\xbc\xe0\x87\x1f\xbc\x01\\V\xbb\xf5\xf0\xf8<3B\xf2<\xb5V\xf3<JJ\x9e;n"\xd0:\xd5h\xcd<\x93\xdf]<\x0eNF<\xf3\xb1\xd0\xbc\xa7\xdc\x9d<\xb8 \xe2\xbaK\xf8\x06=e7\x8a;\xf7\xe2\xce<X\xe0\xef\xbc\x93\x93\xf9\xbc \xdc\x17<A'
)

1990

In [11]:
len(b'\xe0\xea;LD\x89\xbc\xea\x0f\xfb\xbc\xa5\xa5\x1b\xbbC\xb6\xf3<e\xbf\xa2\xbc\x85\x10\xdb<.\xe7\x8e;\x00\xa0\x90\xba\xaa7\xc7\xbc@\x81\x98<.\xe6\xb1<-\xb9\x0e=\xea\xe6o\xbc\xf7\x92\xc5')

58

In [12]:
1990+58

2048

In [15]:
len(b"PARAMS<e\xc0\x97<nR7;\xb7\x02\xca:\xf3(\xb4\xbc'G\xa4\xbc\xb3+a<\xa5\xa9\x86\xbc\xfa\xec\xe5\xbc\xd3\xae\x84<:\xb1\xfa<\xf1\xac\xba\xbc\xb7\xd4\x9e:\xae\x15\xf3\xbc\x13z\x82;^\xec\xe2\xbc\x9cG\xc0;\x011\xc8\xba\xb2\xf7\x01\xbd\x1a\xfb\xcc\xbc\x0e\x14\xaa<@\x85\x81<J_\x90\xbb\x0eW\xbf\xbc|\xcc\xfe\xbc\x91\xde\xd8\xbc\x16\x82\x06=\xca\xd3I;\x9e\xd1\xa7<\xd3\x98\xa9\xbc\xee\x94Z;*4\xed\xbc\xdcF\x08:\xee\x85\xc5<c\xae\x93\xbc\x81\xe60<\x18\xd7m<\xeeON<X\x0ek\xbc\x01\xac\xbb\xba\xa7\xc0\xc8\xbcn+G:\xec\xae\x8f<\xf7\xda\x8f;q7\xad\xbc\xae\x9e\x9d\xbb\xa1f\xc3<S\xef\x8f\xbbw\xaf\x01\xbd\x1cf\xd9<e\x9d\xbd\xbc\xdc\x8aJ\xbc\xe5.\x07\xbd\xc0\x8a\x07\xbcz{\x90\xbc\xe5\xc4\xb1\xbb\xd8r\xfa\xbc\xae\xd0\xac<A\xed\xc3\xbc\x17\x81U\xbc*\xd4&<\xbcK\x1b<3\xf3\x0e=\x00s\x14<\x01\x82\xe7\xbcA,\xce;\xb7J\x8a\xbb\x8ec,\xbc\x0e\x81\x9a<~\xf6\x00=\xe4w\x04=\x9a1\xd9\xbcN\xd3\xcf\xbc\xc7p\xcb<2\xe2\x01=\n\xff\xf7\xbb>D\xb4<\xfc\xac\x10<\xd3\xba\xf2\xbc\xeax\xd7\xbc\xdaM\xb6<[\xdf\r=\xae)\x1e<\xaaU\xb4<\xfc\xbe\xa4<%hz<\xe5\x8b\x9d<J\x94\t;n\xd8\xf7\xbb\xdcM5\xbc\xf6\x9a\t\xbdl\x0f\xf8\xbcS\xa5\x08\xbd\x8a\xef\x0c=\xc5\xc6\xbf<h\xef\xf9\xbc\x81=G\xbb\xf7)\xbf<`\x80\x90<w)\xb4<|hr\xbc\x1a\xc2\xbf<z\xb2\x9d\xbc\x0ei\x16\xbc\xec\xb4\x93\xbcn\xa0W;\\\x87v\xbcw\x8c\x15\xbc\xa5\x8a\x92\xbb\n\x93\x07\xbc\x13\x16|\xbb\xb84v<\x85\xd5\xee<\x0eo\x15<~\x1a\xa5\xbc\x98D\xe8<\xf5\xf0\x9e\xbcG\x11\xa3\xbc\x01\x019\xbb\xaaN\xd1<\xfa\xf7\xdb\xbc\x81\xbap<\x81\xdc\xfd<G\xdc\xb3<^W\xc7\xbc\x03\xd6\x90<\xa5\xcfB\xbbJ\x07/;\x80\xec\x02\xbdw\xb7\xa3<\x11\xd7\xac<\xf5\x14\xc3\xbc\x87\x9f\xb5\xbc\xeei\x0c\xbd<\xa3\x99\xbc>\xca\x96\xbc'w\xa2<\x11}\xea\xbc\x13g\x1d<\x15\x19\x88<Ug\xc4<0=\x90\xbc\xfaU\xc6\xbcU3\x0b=Q\x1f\xfa<\xdc\xd4\xa0\xbanw><Pw\x11=x\xf8_<\xcav9\xbbE\xa45\xbc\xe3\xeb\x08\xbd*K\x07\xbd\xc0\xd2 \xbc\xc1C8\xbc,\x99\xc0\xbc\x9cH\x14<n\xba3<\x93\r\xa4\xba\xa5\x13,\xbc{\x94\t\xbd\xed\xef\x04=%\x00\xa0\xba\xb3\xcf\x81<1\x92\xe5<J\xb2]9\n\xf6\x80\xbcC\xcd\xa4<\xce\x90J<\xca\xfdz\xbb\xd3\xbd\xdf\xbc*\xb2M\xbc.\xb9\x0b=C5\xc9<\xd7F\t=\xee\x16\xe1\xbcE[\xcb\xbc'\x04\xb9\xbc\xa5\x08\xa8\xbb'\x88\t=\xb04\x10\xbd\xbc\xdc$\xbcs\xa6\xad<\xc52}\xbc\x8e\xbc\x94\xbce7\x1e<^_\xd6<\xfaW\x95<\xff7\x11=7A\xa2\xbb7\x9c\x08<\xc1\xe73\xbcq\xdb\xc8<\x93\xc1\xcd\xba\x13\xbb1\xbc\x8e\xf8\x0c\xbc'\xa8\x86\xbc\xc0'\x87<\xee+A\xbcnLj<x\xcf\xfd<\x03\xc4\xc0\xbcJ0\x8e:n\xe6D;\xb0\x99\x93\xbc\x13\x0b\x95;\xa5\x1b\xf6\xbb\xb7\x0f\x10\xbb\xb5D\xfa<\x93Z\xae;\x03\xb9\x0e=<")+len(b'Z\xed<\xce\xf6\xf6<v\xc7\x10\xbdq\x98\xb2<7\x08\xb0\xbc,j\xae<%4\x8a;\xb7\xcd\x97\xb9\xdec\x8e\xbc\x91\x0e\xc4\xbc\xa5L\xae\xbb\n\xae\xc9\xbb\xe5\x90\xe0;\xfc0\x92\xbc!5\xe1\xbc\x9ab\x05=c}\xa5\xbc\xd7J\x0e=z[\xee<\x81N+\xbc\xaa\xda\xb6\xbcJ\x02\x02\xb9s\xa1\xfb\xbc\x95-\x00\xbd\xf8\'\xfc;\xd3\xba\x04\xbc\xea\xd0u\xbc8\xa0\x04=\x8a\x91\x89<\xeb\x86\x04\xbdW\xa6\x8a\xbc\xa5\xd6\x05\xbbC\xa3\xb7\xbc\xb2\x1e\x0b=\n\xd5\xe7\xbc\xf7\x92\x98<\\\x98\xae;\x8eO\xda<\x13\xaa\x0f\xbd2\xe2\x06\xbdE\xf1\x0c=\x17!\x88\xbc\xee\x9b0;\xae\xe4F<\xd5\xb2\x84\xbc\n\x10\xc3;\x8a\xd2\xc3;\xa0[\x8e<\xa1\'\xbc\xbc\xc1\x04\xe2\xbc,\x08\x0f\xbd\x93\xe7v;\xea\xe3u\xbc\x98`\x00\xbdh\x98\xe3\xbc\x9c\xd1\x01\xbc\\\xf7\xc7\xbb{\x14\x12\xbd\xb3W\xbe<\xa5*\r\xbb\xce]F\xbc\xd5\xd2\xfc<\x1c\xae\xce\xbc\xdc\x90+<\xaa\x92=\xbc\x17\x03U<\x13\x1c\xd5\xbb^T\xa4<| "<(^\x00\xbd\xaau\xdd\xbcC\x82\x8f<\xf5m\x02=\xf5\xd6\xb0\xbc\x03\x8a\xe7\xbc\x8e\xd6\n=aL3\xbcJ\xb6\xd2;*L><\x93\xc7:\xbc`$\x1c<\xca8L\xbb\x81\xb7\xcc\xbb%\xc4\xbd;\xbc\xecl\xbcgm\xb5<\x05\xcck<\x82z\x07\xbd\x05\xfa\x0f\xbcag^<8\x03j\xbc\n3\xea\xbb\xfe*\xc0\xbc\'t\x10\xbd.\x85C<\xa3\xba\x87\xbcJ\x1e=<\x93\xdd\x02\xbc\x9c<\xa1;"\x0c\x0b\xbd<\xad\x97<\xca\xfe\xc2\xbb\xdc\x0c2:e\x99W\xbc\x85,\xfd<5\xc1\xaf<\x01)\xdf<\xe01\x19<\n(\x84;\x0eJA<\x00G\x16\xbb\xa5\x87\xe8\xbc4`\x08=EL\r=7\xcf\xa6\xbc\xeez\x8f<Px\x02\xbd\x87"\xb6\xbc\xf0\xaf\x00=\x010\xf0\xba>\x1c\x91<\x05\xf21<\xce\xff\x9e\xbc\x0e\r\t=\xae\x99\xd5<\x93\xb3\xac\xbb\nO\x98\xbb7L7;e_\xd3\xbb\xe3\xba\x94\xbc\xc7:\xd2<\x97\x07/\xbc\x83\xfa\xf1\xbc|p\x16<\x00\xa1\x96\xbc\xfa\xa4\xdf<\xa5\xc9\x96;e\x02I\xbcE\xc4{\xbc\x03~\x02=*\xf9\xeb<\xb81\xee\xb9e\xc1\xf4\xbc\x15\xb6\xea\xbc\xf3\xf4M<S&\xc9<\x97\xff\x8d\xbc\xdcC\xd2\xbb\x13Ut\xbc\x03R\xae<\xe3\x8a\x01\xbd\x93\xfa\x89<\xf3.\x08\xbd\xe5\xb1\x9c\xbcnC\xd79\x9e\xe7\xdb<\x8eJ:<\xfc\x07\x1e\xbc\x8eu\x8e\xbc\xd8\xf6\xfb\xbcZI\x04\xbds)\x1e<%\xc76<M)\x05=\\\x9d\xd2\xbc|\xa7\xa6\xbcc\x9a\x91\xbc\xf58\xc7\xbc\xac+\xa8\xbc\xbe\x08\x8e<\xdc \xfb\xbb\x1c\x14\xa5;J\x078<N\xfaT<j\xdcP<\x98\x85c\xbc\x80\x97#\xbb\x93F\n=n\x15\x87\xba\x05\xb1\x97\xbc\x01\xd9\xe4\xbcNo)<\x13b}<N\x98J<E\xe5)<\x8c\xba\xff\xbc\n\x12\xbb\xbcS\x8c\x92\xbbz\xdd\xa9<\x00@\x93\xb9\x9e4\xa0<\x1a\xa6\r\xbd\x8a\xf1G\xbc\x15\xb2\xff\xbc \x9f\x97\xbc\xfc\xcc\xa9<\x1c\x9bj\xbc\x15\x19\xd0<\xe1@\xe2<\xcb-\x07\xbdJ\xb2=\xb8\x95\xb6\xfb\xbc\x189j<j:\x8b<G,\x08=\xc7\xc9\x86<VR\t=\n\xdb\x02\xbcc\x90\xdf<\xdc"\x1d<\xb7\xcb?<\xc5\n\x10\xbc\x9a5\xae\xbcu\xc1\xd7<\x13DJ\xbb|\xe7\x83<\xb3\xef\xd6\xbcJq\xe3\xba\xaa\xdc\x1b\xbc\x1c\x8c\xa0<\x930\xdd<\xad\xb5\x02\xbd\x8e_V<LD\xca\xbc\xcc\x0b\xfd<\x05\xa6\x0b\xbcc\xd4\xfc<nGv\xbc\xd7,,\xbc\xdc\xb6u6\xdc\x92?;\x83?\xd8<nJ\'<zd\xb2\xbcuT\xbe\xbc\xa7\xf2\x0e=Z\xe6\x98<\xb7y\xa6\xbbH\x1c\xf8\xbc\xdcOP\xbc\xbc\xf1\x8c<\xca\xb8\xc3\xbb\x18\xf1z\xbcA\xef\xb2;\xd7\xa4"\xbc\x1c\xf1G<7\x0b\xaf\xbb\xaa\xd8?<\xf0!\x00=\\\xbf?\xbcW\x198\xbc\x9eF\xd2\xbcj\x8aI\xbcQ\\\xcc<\xae\xcc\xf1<%\xfe\xc0\xba\xb3?\x8e\xbc\x1e\xc6\xf3\xbc\xc6[\x01=\x1coQ<\xec\xa0\xe2\xbcX\x8f\xf4<<\x98;<\xdc\r\xec\xbaW\x03\xaa<Ef\x81<\xc1\xd1\xb2\xbc\xb7\xba\xaf;\x13\x04j<\xd7\xec\x86<\x8f\xf9\x0e=\xccf\xbe\xbc\x01\xa0\xbb\xb8.\x06\xb6;}<\x07=J\xac\x12;\x01\x0f\xfe:n\x04Z\xbb\\\x98\x07\xbbl\xb2\xd6\xbc\xf5\xd9\xad\xbc\xdc\xd0\x8d\xba\xdb\x85\x0c=3L\x98\xbcn\xaf\xf59\xa1@Y\xbc.\xe0p\xbc1,\xc5\xbcCU\xf9<\xd3O%<\x8c\x06\x88\xbcm\xb9\x08=\xe7a\xc3<\xf3\x1f\xb7\xbc\x95\xa8\xb0\xbc!$\x7f\xbc:{\r=\xe3\xeb\xa0\xbc\x9c{\xb9\xbc\xc7\xc7\x81\xbc\xae')

2048

In [16]:
len(b'PARAMS\x80\x04\x95\x87\x00\x00\x00\x00\x00\x00\x00]\x94(\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x80M\x10\x03\x86\x94h\x04\x8c\x05dtype\x94\x93\x94\x8c\x02f4\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B\x00 \x06\x00\\ \\\xbc \x81\x86\xbc\xe3)\xe3\xbcN\\\xe9\xbc\xbc\xc1\xb6\xbc\xb0\xff\x0c\xbd\xfc\x00E\xbc\x98\x0e\xf3\xbcnj\xab;r\xf9\x0f\xbd\x9e\x1a\x91\xbc\x01Yr;\xber\xdd<%\xbb\x03;J\xee\x92\xb9j.\x0c=\x81@\xbf;%\x9bj\xba(\x8d\xfe\xbc%\xa8\x92<sv\x8a\xbcJ\xf7\x19\xbc%\xcd\x819\x9e@\xef<e\x05\x0c\xbc\x1c\x94\x94<\xc1\x01\xc4\xbb\x16\xad\x0f\xbd\x88\xb8\xfd\xbc\xe7\x07\xb6<\x8a}\x88\xbc\xab\xb1\x00\xbd\x01\xdb`\xbc\xa5\x8fw\xbc\n\x1e\xe5<\x93\xe6\x02<J^\xe0\xbb%\xabF\xbb8\xe1\xe2\xbb\x83Q\n\xbdg7\x03=J`H\xba\x93\xed\xb2\xbc\x84\xde\x08\xbdE\xf7\x03\xbd\x01\xcb6;wR\x8e;n\xe1\xa4:~E\x92<\xb7\x17\x03\xba\x13\x19J\xbb\x00L\x97\xbc\xca\x05\xf0\xbbJ\xcau\xbc\x85\xdf=\xbc\x05\x91\xd6\xbc%\xd4M\xbbW\x90\xa0<w+\x10\xbcnCl<\xdcG\x88;Q\r\xcc<e[$\xbc\xa8M\xdf<3\x16\x1a<\xe52a<\xaa\x1a\xde\xbcD!\x00=j\xa1P<\x01BX<\x17\xcb\xa3\xbc7\x87H\xbb\x93\x8a\x08<\xf8\xfd\xe7\xbbg\x98\x96\xbc\xfa\xfc\x02\xbd\xca\xdal\xbb\xf3uJ\xbc\x9a\xd1\xc1\xbc\xe5\x93\xab;\xf8\x7f\\\xbc\x8aZ\x89\xbc \x89\x07<\x13\xd3\xd3<s\xa3:\xbcE\xa7u<\x0c\x9f\xf1\xbc@\xd7\x97;\x9bv\x0f=<\x9d\xaa\xbc\x03\x18\xa1\xbce\xf0\x9f\xbc\xd7\x10\x12<\xa0B)<\xae\x8a\x0f\xbdq\x1e\xe7\xbc\xdd^\x01\xbdU\x99\xe5<r\xce\t\xbdh\x11\xdd\xbc\xe3\x89\x86<\n\x97\x83;~]\xcb\xbc\xf0\x01\xa5<\xc1\xd4\xe6;\n\xc8\xa6<JW\x9d;e\x87\xa6<<~\x8a<3 l\xbc\xfc\xb8\x91\xbc\x1a\x9d\xd5\xbcn\x8f\xe9\xba3\xfb\x08\xbd7]\x1f;\xb5\x01\xea<|\x18><n&L;\x01W\xd4<\x8a\xe0\xf8\xbcn<\x1b;\xbeV\xbe\xbcE\xb4]\xbc\xf1%\xd5\xbc\x9a\x1e\x0f\xbd|p\xc9\xbc`\xd0\x11\xbd\\\x81\xa5\xbb\x81\xfdR<LO\xb1\xbc\xf8L{\xbc\xa5/T\xbcGt\xa8\xbcE\x05\x1e<\xac\xdf\xc7\xbc\xaaq`<A\x91\xd2<\x98G\xef<\xe0\xd6\x94<\x81#\xd6;\xb7\xe3\x8e\xbaj\xde\x0e=\x00\x92\x87;\xc0\xf4\x85;\xdcT\x99;\x88u\xdc<Xkv\xbc\x820\x11=\x0e\xc9\xe2\xbcJ\x86\xe2\xba\x8a\x10\x04\xbd\xb7\x859\xb9\xae\xa4\xac\xbc_\xbf\x06=\x9cc\x0b<\xae\xf2\x80\xbbnd\x81\xbb\xb8}\xee\xba%\x11\x81\xbb\xe2\x7f\x02\xbd\n\x8e\xfe<\x9a\x8c\x0c=\x817^\xbb\x13\x91\xb5\xbc\x15\xda\x98<\xb7[\x96:n\xb2g<G\x18\xa0\xbc\x00\x08\r<\xca\xad\xab\xbc~\x08\xbb\xbc\xd7K\xa8\xbcw\xacJ\xbc\xba\x92\x9d\xbc\xc8E\xf6\xbcS)\xd8;?\x04\x12\xbd\x1c\xdc\'\xbcE\x95\xdb<Z\x00\xc9<\xa7\xaa\x03=\xa5E9;\xee\xad=\xbc8\xe5\xf5;J\xc2\xfb:.*\xeb;\\\xe7\xdc;\xce\x066<\xee\x08`<\x8a@\x83\xbb%\\\x98<\x1c\xc9\xbe<\xae\x1b\xa0<\xa1\x15\xf6\xbc\xba]\n=\xdc!\xb8\xbc\xa6\xc1\x03=\xa1\xea\xfd<\x9a\xec\x96\xbc\x13\x81\xc3;uA\xeb\xbc>\x96\xb2<\x10\x90\xa6\xbcLm\xcb\xbc8\x11\xe2<\x01Q.\xbc\xb8\xf8\xdc<\xf3\xa9\xe5<:\x83\xcb<@d\x8c\xbcu%\xb2\xbc\xaa\xce6\xbc\xb0N\x90\xbc\x8a,0<\xba\x9a\xd1\xbc\xbc\xf0\x00<\xac\x1e\xe0<:\xef\x08=\x0e\xee\xd1\xbcG\xa8\x01\xbd\xce\xde\x10\xbc\xe5\xa3g<\xf7a\xa0;\x88\x8b\x0c\xbd\x9c \xde;EL]\xbcn\x9aT<\xb7;\xbf\xbb\xee\x1e\x7f;W\x84\xd5\xbc%\xf4\x0f\xbb!\x18\xd0<\x93\xbfC\xbb\x10\xff\x9a\xbc>/\xe9\xbc=#\x00\xbd\xd3?E<`\xd4\xa7<\xe6\x8e\x10=\'{\xbf\xbc\x9cv\xfd<\xae\xaa\xf1\xbb\xea\xa5\r=J6\x82\xbb\x84\xe0\x10=\xa1W\xf5\xbcn\n\xfa<\x97\x11\x00<\xd1b\xf4<\x9a\x9b\xcc<\x9em\x0f\xbd\xc3\x1b\x06=\xe6*\x08\xbd\xcaK\x99<\xb3Ey\xbc\xee\nL;nn\xa3:\x97\xd0\n=^\xa6\x84<\xca\xd8\xb4<\xa5d\xf6\xbc\x97\xca\t=\x93\xbe\xf9\xbb\xe5M\x93;*\x17\x04=\x15\xc7\xd6<*TS\xbc\xbaJ\xb5\xbc\xa51\xf7\xbb8Vb;\xa7\xe8\xaa<L~\xe2\xbc[\x81\x11\xbd\xd5\x10\xa3\xbc\x1c\x19R\xbc\xbaj\xd3<\x11\xae\xc4\xbc74B<\xee0h;\x8eX\x11\xbd\xc0}\xa6<\xca\xa5\xb1\xbb\xf8\x0b\xee\xbbeO\x94<J\xead:\x1e\xa5\xfe\xbc\xc0V\x8b\xbb|9*<q\xaa\xf5\xbc\xb3%0<\x04\xb5\x08=\xbe\x8f\x03\xbd\x80r%\xbb\x0c>\x91\xbc\x13\xde\xb0\xbb\xc1\x8b\xd2\xbc\xaa\xe2\x86<n\xd9\xdb<S\x10\x1f\xbcd\xeb\x01\xbdn@`\xbcn;\x98\xb8Z\x0e\xc2<\xdc\xb6\xd07E\xe0\x0b\xbd\\k\n<\xd2\xd6\t\xbd\xc3\xa2\xb2<@\x88\x00<\xc1\xbe\xe6\xbcS|\xfd;O/\x05=\x95\xec\xc8<\xbeY\xaf\xbcQ\x7f\xe7\xbc\xe5\xed\x89\xbcJ\xd2]\xb9%\xd6\xee\xbb7J\x12\xbb*\xa7~\xbc\xc1\x01Q<\xa1\x9c5\xbc.\xed\xc9;\n\xaf\xe6\xbc|\xe2\xba\xbc\xae{\x97<e\x9e\xf5;S8s\xbc\xe7\xc0\xcf\xbcU\xec\x01=\xca\x14\'\xbc%e\xd8;\x1e\xe1\xb3<\x1c\x18K\xbc\x83\x01\xfe<S\xbc\xb0<\xf8\xcak\xbc\x93\x89\x81:\xba\x04\x91\xbc\xd0\x08\x06=N?\xd8\xbc.8\xd8<\x97\x15\x9c\xbcN\xb2\xaa\xbcS\xbbD\xbcg\xf1\xc5<\xae\x8fp<\\\xf2\x01\xbdu\x0e\xca\xbc\x98h\xef<\x13\x0cB\xbc\xc5\x82\x9d\xbcW\xe3-\xbc\xdc\xe0P\xbaJ\xa7\xb0\xbcj\xa9\x10=\x93-a<\xd5\x9f\x8f<NO\x02<sH\x0e\xbd\x01\xc3^<.\r\xaa\xbc\xf3\xe1@<\x11c\xf4<\xb0\xd1\x8a<74\x93\xbc\xdcb\x16;\xd3\xf9N<|\xab\xf3<\xf3\x0b\xdd\xbc\xe5g\xb6\xbc\x8e\x9b\xd7<\xa3\xe3\x00\xbd\xb7\x19\x0f\xbb\xb7O\xa8<\xf7J\x11<ni\xed<\x05\xd1\t\xbc\xdc\x89\xbf\xbb\x1ec\xff\xbc7\xd8>\xbc\xce\xcc\xca\xbc\xb7=\x87\xba\xee\xbb\x1c\xbc\x0c{\xe8<\x84\x9e\x04\xbdJ\xbe\xce;\x1a\xeb\xb2\xbcu}\xcc<S\xdb\x8c\xbc\xf0\x19\xaa\xbcNre<n\xf2N;u\xea\xe8\xbc\n7\x89\xbb\x910\xb9\xbcJ\xee\x1f\xbb\xdc\x91\x96:\xf8D\xe1;E\x11\x12\xbc\xc3J\xed\xbcn\xa2L\xbb\xee\xcc\x94<>\x94\xfd<\x8c\x89\x03=\xa7|\x82<\xc5\xeb><\xa5L\xa2\xbc\xca\x83}<s1\xae<(\xc2\xf0<\\\x9d\x87\xbb\xae:\x99<\xf3\xaeI<a40\xbc\xf7\xeb\x84;{\xad\x0f=a\xfc\xdb<\xf0\x8b\x8f\xbc!\x0f\xdd<\xba\x0f\xba<\xd5u\x9a\xbc\xa3\xd9\x99\xbc\xf7\x8c\x11\xbd\xaab\x95\xbc\'2\x85<%\xed!\xbb\xf3\xa6\t\xbc\xb3+\xae<\xc1\xc2\xfb;\xca2\xb3\xbb\xd7CT<N\xfdi<\x80]\x90\xbcnp\xbf\xba\x13{|\xbc\xf7q\xb2<n[\xc57!\x99S\xbc\xe0\x87\x1f\xbc\x01\\V\xbb\xf5\xf0\xf8<3B\xf2<\xb5V\xf3<JJ\x9e;n"\xd0:\xd5h\xcd<\x93\xdf]<\x0eNF<\xf3\xb1\xd0\xbc\xa7\xdc\x9d<\xb8 \xe2\xbaK\xf8\x06=e7\x8a;\xf7\xe2\xce<X\xe0\xef\xbc\x93\x93\xf9\xbc \xdc\x17<A\xe0\xea;LD\x89\xbc\xea\x0f\xfb\xbc\xa5\xa5\x1b\xbbC\xb6\xf3<e\xbf\xa2\xbc\x85\x10\xdb<.\xe7\x8e;\x00\xa0\x90\xba\xaa7\xc7\xbc@\x81\x98<.\xe6\xb1<-\xb9\x0e=\xea\xe6o\xbc\xf7\x92\xc5'
)

2048